# CNN Max Pooling To Standardize Input Sizes

__Idea__: The images have very different resolutions so passing the larger images to the same CNN Global Average Pooling filters as the smaller images will produce very different feature maps. To counteract this, we first pass the larger images through normal CNN max-pooling to reduce them to similar dimensionality.

In [1]:
# Required imports:

from collections import defaultdict
import os
import sys

from tensorflow import keras
from tensorflow.keras import layers
import yaml

sys.path.append(os.path.join("..", "code"))
from tif_files import *
from utils import open_swe_file


DIR = os.getcwd()
DATA_DIR = os.path.join(os.path.dirname(DIR), "data")
SAMPLE_DIR = os.path.join(DATA_DIR, "sample_images")
DEM_DIR = os.path.join(DATA_DIR, "dem")

2022-11-21 18:24:20.811708: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Calculate shapes of all different input images:

In [2]:
shapes = defaultdict(dict)

for filename in os.listdir(SAMPLE_DIR):
    fp = os.path.join(SAMPLE_DIR, filename)
    if filename.endswith(".tif"):
        tif = TifFile(fp)
        shapes[tif.band][tif.gage] = tif.shape
    elif filename.endswith(".npy"):
        assert "swe" in filename.lower()
        arr = open_swe_file(fp)
        gage = re.findall("\d{8}", filename)[0]
        shapes["swe"][gage] = arr.shape
        
for filename in os.listdir(DEM_DIR):
    fp = os.path.join(DEM_DIR, filename)
    if fp.endswith(".tif"):
        tif = TifFile(fp)
        shapes[tif.band][tif.gage] = tif.shape

In [3]:
shapes

defaultdict(dict,
            {'et': {'11266500': (79, 106),
              '11208000': (41, 53),
              '11185500': (215, 111),
              '11202710': (62, 51),
              '11189500': (180, 97),
              '11318500': (26, 97),
              '11402000': (50, 119)},
             'temp': {'11402000': (3, 6),
              '11208000': (2, 3),
              '11185500': (10, 6),
              '11202710': (4, 3),
              '11189500': (8, 5),
              '11318500': (2, 5),
              '11266500': (5, 5)},
             'swe': {'11202710': (240, 198),
              '11208000': (158, 208),
              '11189500': (709, 380),
              '11266500': (309, 419),
              '11185500': (848, 436),
              '11402000': (195, 467),
              '11318500': (101, 382)},
             'precip': {'11318500': (2, 5),
              '11202710': (4, 3),
              '11208000': (2, 3),
              '11189500': (8, 5),
              '11266500': (5, 5),
              '1

 Calculate averages for each band:

In [4]:
avg = dict()
for band, data in shapes.items():
    row_avg = np.mean([v[0] for v in data.values()])
    col_avg = np.mean([v[1] for v in data.values()])
    avg[band] = [row_avg, col_avg]
int_avg = {k: [int(v[0]), int(v[1])] for k, v in avg.items()}
with open(os.path.join(DATA_DIR, "avg_img_sizes.yaml"), "w") as f:
    yaml.safe_dump(int_avg, f)
int_avg

{'et': [93, 90],
 'temp': [4, 4],
 'swe': [365, 355],
 'precip': [4, 4],
 'dem': [1425, 1386]}

Since 'temp' and 'precip' are similar sizes and the smallest, we need CNNs to get the other inputs down to a similar scale.

## ET CNN

In [5]:
def et_cnn(input_shape: tuple, observation_period: int, activation: str = "relu"):
    
    inputs = keras.Input(shape=(observation_period, input_shape[0], input_shape[1], 1), 
                         batch_size=None, name="ET_inputs")
    
    # First convolutional layer:
    conv_2d_layer = layers.Conv2D(filters=1, kernel_size=(3, 3), 
                                  strides=(2, 2), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer, name=f"ET_conv2d_0")(inputs)
    pooling_layer = layers.MaxPooling2D()
    outputs = layers.TimeDistributed(pooling_layer, name=f"ET_max_pooling_0")(x)
    
    # Second convolutional layer:
    conv_2d_layer1 = layers.Conv2D(filters=1, kernel_size=(2, 2), 
                                   strides=(1, 1), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer1, name="ET_conv2d_1")(outputs)
    pooling_layer1 = layers.MaxPooling2D()
    outputs1 = layers.TimeDistributed(pooling_layer1, name=f"ET_max_pooling_1")(x)

    return (inputs, outputs1)

In [6]:
# Print model summary for average size input:
mdl = keras.Model(*et_cnn(int_avg["et"], 7))
mdl.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ET_inputs (InputLayer)      [(None, 7, 93, 90, 1)]    0         
                                                                 
 ET_conv2d_0 (TimeDistribute  (None, 7, 46, 44, 1)     10        
 d)                                                              
                                                                 
 ET_max_pooling_0 (TimeDistr  (None, 7, 23, 22, 1)     0         
 ibuted)                                                         
                                                                 
 ET_conv2d_1 (TimeDistribute  (None, 7, 22, 21, 1)     5         
 d)                                                              
                                                                 
 ET_max_pooling_1 (TimeDistr  (None, 7, 11, 10, 1)     0         
 ibuted)                                                     

2022-11-21 18:24:28.100191: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
# Print model summaries for all shape inputs:
for gage, shape in shapes["et"].items():
    mdl = keras.Model(*et_cnn(shape, 7))
    mdl.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ET_inputs (InputLayer)      [(None, 7, 79, 106, 1)]   0         
                                                                 
 ET_conv2d_0 (TimeDistribute  (None, 7, 39, 52, 1)     10        
 d)                                                              
                                                                 
 ET_max_pooling_0 (TimeDistr  (None, 7, 19, 26, 1)     0         
 ibuted)                                                         
                                                                 
 ET_conv2d_1 (TimeDistribute  (None, 7, 18, 25, 1)     5         
 d)                                                              
                                                                 
 ET_max_pooling_1 (TimeDistr  (None, 7, 9, 12, 1)      0         
 ibuted)                                                   

## SWE CNN

In [8]:
def swe_cnn(input_shape: tuple, observation_period: int, activation: str = "relu"):
    
    inputs = keras.Input(shape=(observation_period, input_shape[0], input_shape[1], 1), 
                         batch_size=None, name="SWE_inputs")
    
    # First convolutional layer:
    conv_2d_layer = layers.Conv2D(filters=1, kernel_size=(5, 5), 
                                  strides=(3, 3), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer, name=f"SWE_conv2d_0")(inputs)
    pooling_layer = layers.MaxPooling2D()
    outputs = layers.TimeDistributed(pooling_layer, name=f"SWE_max_pooling_0")(x)
    
    # Second convolutional layer:
    conv_2d_layer1 = layers.Conv2D(filters=1, kernel_size=(3, 3), 
                                   strides=(2, 2), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer1, name="SWE_conv2d_1")(outputs)
    pooling_layer1 = layers.MaxPooling2D()
    outputs1 = layers.TimeDistributed(pooling_layer1, name=f"SWE_max_pooling_1")(x)

    return (inputs, outputs1)

In [9]:
# Print model summary for average size input:
mdl = keras.Model(*swe_cnn(int_avg["swe"], 12))
mdl.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SWE_inputs (InputLayer)     [(None, 12, 365, 355, 1)  0         
                             ]                                   
                                                                 
 SWE_conv2d_0 (TimeDistribut  (None, 12, 121, 117, 1)  26        
 ed)                                                             
                                                                 
 SWE_max_pooling_0 (TimeDist  (None, 12, 60, 58, 1)    0         
 ributed)                                                        
                                                                 
 SWE_conv2d_1 (TimeDistribut  (None, 12, 29, 28, 1)    10        
 ed)                                                             
                                                                 
 SWE_max_pooling_1 (TimeDist  (None, 12, 14, 14, 1)    0   

In [10]:
# Print model summaries for all shape inputs:
for gage, shape in shapes["swe"].items():
    mdl = keras.Model(*swe_cnn(shape, 12))
    mdl.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 SWE_inputs (InputLayer)     [(None, 12, 240, 198, 1)  0         
                             ]                                   
                                                                 
 SWE_conv2d_0 (TimeDistribut  (None, 12, 79, 65, 1)    26        
 ed)                                                             
                                                                 
 SWE_max_pooling_0 (TimeDist  (None, 12, 39, 32, 1)    0         
 ributed)                                                        
                                                                 
 SWE_conv2d_1 (TimeDistribut  (None, 12, 19, 15, 1)    10        
 ed)                                                             
                                                                 
 SWE_max_pooling_1 (TimeDist  (None, 12, 9, 7, 1)      0   

## DEM CNN

In [11]:
def dem_cnn(input_shape: tuple, observation_period: int, activation: str = "relu"):
    
    inputs = keras.Input(shape=(observation_period, input_shape[0], input_shape[1], 1), 
                         batch_size=None, name="DEM_inputs")
    
    # First convolutional layer:
    conv_2d_layer = layers.Conv2D(filters=1, kernel_size=(5, 5), 
                                  strides=(3, 3), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer, name="DEM_conv2d_0")(inputs)
    pooling_layer = layers.MaxPooling2D()
    outputs = layers.TimeDistributed(pooling_layer, name=f"DEM_max_pooling_0")(x)

    # Second convolutional layer:
    conv_2d_layer1 = layers.Conv2D(filters=1, kernel_size=(4, 4), 
                                   strides=(2, 2), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer1, name="DEM_conv2d_1")(outputs)
    pooling_layer1 = layers.MaxPooling2D()
    outputs1 = layers.TimeDistributed(pooling_layer1, name=f"DEM_max_pooling_1")(x)
    
    # Third convolutional layer:
    conv_2d_layer2 = layers.Conv2D(filters=1, kernel_size=(3, 3), 
                                   strides=(2, 2), activation=activation)
    x = layers.TimeDistributed(conv_2d_layer1, name="DEM_conv2d_2")(outputs1)
    pooling_layer2 = layers.MaxPooling2D()
    outputs2 = layers.TimeDistributed(pooling_layer2, name=f"DEM_max_pooling_2")(x)

    return (inputs, outputs2)

In [12]:
# Print model summary for average size input:
mdl = keras.Model(*dem_cnn(int_avg["dem"], 1))
mdl.summary()

Model: "model_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 DEM_inputs (InputLayer)     [(None, 1, 1425, 1386, 1  0         
                             )]                                  
                                                                 
 DEM_conv2d_0 (TimeDistribut  (None, 1, 474, 461, 1)   26        
 ed)                                                             
                                                                 
 DEM_max_pooling_0 (TimeDist  (None, 1, 237, 230, 1)   0         
 ributed)                                                        
                                                                 
 DEM_conv2d_1 (TimeDistribut  (None, 1, 117, 114, 1)   17        
 ed)                                                             
                                                                 
 DEM_max_pooling_1 (TimeDist  (None, 1, 58, 57, 1)     0  

In [13]:
# Print model summaries for all shape inputs:
for gage, shape in shapes["dem"].items():
    mdl = keras.Model(*dem_cnn(shape, 1))
    mdl.summary()

Model: "model_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 DEM_inputs (InputLayer)     [(None, 1, 3305, 1697, 1  0         
                             )]                                  
                                                                 
 DEM_conv2d_0 (TimeDistribut  (None, 1, 1101, 565, 1)  26        
 ed)                                                             
                                                                 
 DEM_max_pooling_0 (TimeDist  (None, 1, 550, 282, 1)   0         
 ributed)                                                        
                                                                 
 DEM_conv2d_1 (TimeDistribut  (None, 1, 274, 140, 1)   17        
 ed)                                                             
                                                                 
 DEM_max_pooling_1 (TimeDist  (None, 1, 137, 70, 1)    0  